In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from dotenv import dotenv_values
import time
import os

In [ ]:
options = webdriver.FirefoxOptions()
options.add_argument("-detach")
driver = webdriver.Firefox(options=options)

In [ ]:
BASE_URL = "https://course.testpad.chitkara.edu.in"
SOLUTIONS_FOLDER = "Testpad-Solutions2"
COURSE_NAME = "24CSE0208-Data Structures using Object Oriented Programming-2024-CSE-3Sem"

In [ ]:
driver.get(BASE_URL)

wait = WebDriverWait(driver, 10)
wait.until(EC.frame_to_be_available_and_switch_to_it((By.ID, "loginIframe")))

In [ ]:
testpad_credentials = dotenv_values(".env")

email_element = driver.find_element(By.ID, "email")
email_element.send_keys(testpad_credentials["TESTPAD_EMAIL"])

password_element = driver.find_element(By.ID, "password")
password_element.send_keys(testpad_credentials["TESTPAD_PASSWORD"])

submit_button = driver.find_element(By.ID, "submit")
submit_button.click()

driver.switch_to.parent_frame()

In [ ]:
def wait_for_loader_disappearance(): wait.until(EC.invisibility_of_element_located(
    (By.ID, "loader-container")))

wait.until(EC.visibility_of_all_elements_located(
    (By.CLASS_NAME, "learning-module-card")))

learning_module_card = driver.find_element(
    By.XPATH, f"//a[text()='{COURSE_NAME}']").find_element(By.XPATH, "../..")

learning_module_href = learning_module_card.find_element(
    By.CSS_SELECTOR,
    "[class='icons-link-container tooltip-up'][data-customtooltip='Learning Content']"
)

wait_for_loader_disappearance()
wait.until(EC.element_to_be_clickable(learning_module_href)).click()

In [ ]:
"""
Wait for subtopics to show up
"""
wait.until(EC.presence_of_all_elements_located(
    (By.CLASS_NAME, "course-topics")))

course_topics = driver.find_elements(
    By.CLASS_NAME, "course-topics")

no_of_topics = len(course_topics)
if no_of_topics == 0:
    raise ValueError("No course topic showing!")

print(f"Number of subtopics- {no_of_topics}")

wait_for_loader_disappearance()

original_window = driver.current_window_handle

In [ ]:
CORRECT_ANSWER_ICON = "/icons/check1.svg"
EXTENSION_MAPPING = {
    '.c': "C",
    ".java": "Java",
    ".cpp": "C++",
    ".py": "Python",
    ".sql": "SQL"
}

In [ ]:
def wait_until_stable(element, timeout=5, interval=0.2):
    end_time = time.time() + timeout
    last_rect = None

    while time.time() < end_time:
        rect = driver.execute_script(
            "return arguments[0].getBoundingClientRect().toJSON();", element
        )
        if rect == last_rect:
            return True
        last_rect = rect
        time.sleep(interval)
    return False

In [ ]:
def complete_coding_question(question_file_path):
        coding_question_file_content = open(question_file_path).read()

        dropdown_language_selected = wait.until(EC.visibility_of_element_located(
            (By.CSS_SELECTOR, "button[data-id='progLang']")))
        codefile_extension = question_file_path[question_file_path.rindex(
            "."):]
        
        if EXTENSION_MAPPING[codefile_extension] != dropdown_language_selected.text:
            for attempt in range(2):
                try:
                    wait_for_loader_disappearance()
                    wait_until_stable(dropdown_language_selected)
                    driver.execute_script(
                        "arguments[0].click()", dropdown_language_selected)
                    # time.sleep(1)  # allow dropdown animation

                    programming_languages_ul = WebDriverWait(driver, 3).until(
                        EC.visibility_of_any_elements_located(
                            (By.CSS_SELECTOR, "ul[class='dropdown-menu inner show']"))
                    )
                    programming_languages_ul = [item for item in programming_languages_ul if item.is_displayed()][0]
                    programming_languages = programming_languages_ul.find_elements(
                        By.TAG_NAME, "li")

                    for language in programming_languages:
                        language_text = language.find_element(By.CSS_SELECTOR, "a > span").text
                        if language_text == EXTENSION_MAPPING[codefile_extension]:
                            language.click()
                            break
                    break  # success → exit retry loop

                except TimeoutException as e:
                    print(f"Click attempt {attempt+1} failed: {e}")

                    # If dropdown stayed open, close it to reset
                    dropdown_menu_show_status = driver.find_element(
                        By.CSS_SELECTOR,
                        "div.dropdown.bootstrap-select.form-control.form-control-sm.select-tag-for-lang-change"
                    )
                    if "show" in dropdown_menu_show_status.get_attribute("class"):
                        driver.execute_script(
                            "arguments[0].click()", dropdown_language_selected)
                        WebDriverWait(driver, 2).until(EC.invisibility_of_element(
                            dropdown_menu_show_status))

                    time.sleep(1)
            else:
                raise ValueError("Unable to select language after 2 retries ❌")
            

        # dropdown_language_selected = driver.find_elements(
        #     By.CLASS_NAME, "filter-option-inner-inner")
        # dropdown_language_selected = [
        #     item for item in dropdown_language_selected if item.is_displayed()][0]
        
        # if EXTENSION_MAPPING[codefile_extension] != dropdown_language_selected.text:
        #      raise ValueError("Correct language was not selected!")

        wait.until(
            EC.presence_of_element_located((By.CLASS_NAME, "CodeMirror"))
        )

        driver.execute_script("const cm = document.getElementsByClassName('CodeMirror cm-s-default')[0].CodeMirror;" +
                              f"cm.setValue({coding_question_file_content!r});")
        
        submit_code_button = driver.find_element(By.ID, "executeProgram")
        wait_for_loader_disappearance()
        submit_code_button.click()

        try:
            test_cases = wait.until(EC.visibility_of_all_elements_located(
            (By.CLASS_NAME, "test-cases")))
        except TimeoutException:
            print("Submit button wasn't clicked the first time! Clicking again")
            driver.execute_script("arguments[0].click()", submit_code_button)
            test_cases = wait.until(EC.visibility_of_all_elements_located(
                 (By.CLASS_NAME, "test-cases")))

        test_cases_passed = driver.find_elements(
            By.CSS_SELECTOR, f".test-cases > img[src='{CORRECT_ANSWER_ICON}']")
        if len(test_cases_passed) != len(test_cases):
            raise ValueError("All test cases did not pass ❌")
        else:
            print("Question successfully attempted ✅\n")

In [ ]:
def complete_textarea_question(question_file_path):
        mq_question_file_content = open(question_file_path).read()

        questions = wait.until(EC.visibility_of_all_elements_located((
            By.CSS_SELECTOR, "#compileTestcases > [class='col-md-12 test_cases']")))
        answers = mq_question_file_content.strip().split("\n\n\n")
        if len(questions) != len(answers):
            raise ValueError(f"{len(questions)} questions, {len(answers)} answers")

        for i in range(len(questions)):
            driver.execute_script(f"const textarea = document.querySelector('#txtOutput{i}');" +
                                    f"textarea.value = {answers[i]!r};")
        
        for attempt in range(2):
            try:
                wait_for_loader_disappearance()
                execute_program_button = driver.find_element(
                    By.ID, "executeProgram")
                wait_until_stable(execute_program_button)
                wait.until(EC.element_to_be_clickable(
                    execute_program_button)).click()
                outcomes = WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located(
                    (By.CSS_SELECTOR, "label.test_case_result > img.correct_result")))
                break
            except Exception as e:
                print(f"Click attempt {attempt+1} failed: {e}")
                time.sleep(1)
        else:
            raise RuntimeError("Submit button could not be clicked after 3 tries ❌")
        
        no_of_correct_answers = 0
        for outcome in outcomes:
            outcome_image_type = outcome.get_attribute("src")
            if outcome_image_type == BASE_URL + CORRECT_ANSWER_ICON:
                no_of_correct_answers += 1     

        if no_of_correct_answers == len(questions):
            print("Question successfully attempted ✅\n")
        else:
            raise ValueError(
                f"{no_of_correct_answers}/{len(questions)} correct answers ❌\n")

In [ ]:
def complete_mcq_question(question_file_path):
    mcq_question_file_content = open(question_file_path).read()

    mcq_options = wait.until(EC.visibility_of_all_elements_located((
        By.CSS_SELECTOR, "[class='col-12 mt-2 div-radio-btn-choose-answer']")))
    mcq_answer = int(mcq_question_file_content)
    wait_for_loader_disappearance()
    mcq_options[mcq_answer -
                1].find_element(By.CLASS_NAME, "choose-answer-checkmark").click()

    print(f"Correct answer- option {mcq_answer}")

    for attempt in range(2):
        try:
            submit_button = driver.find_element(By.CSS_SELECTOR, "input#executeMCQ")
            wait_for_loader_disappearance()
            submit_button.click()
            wait.until(EC.element_to_be_clickable(submit_button))

            WebDriverWait(driver, 5).until(EC.visibility_of_element_located(
                (By.CSS_SELECTOR, "span.text-success")))
            break
        except TimeoutException as e:
            print(f"Click attempt {attempt+1} failed: {e}")
            time.sleep(1)
    else:
        raise ValueError("Unable to click submit button after 3 retries ❌")

In [ ]:
def complete_tutorial():
    try:
        code_submision_elements = wait.until(EC.visibility_of_all_elements_located((
        By.CLASS_NAME, "code_submision")))
    except:
        wait_for_loader_disappearance()
        code_submision_elements = driver.find_elements(
            By.CLASS_NAME, "code_submision")

    for code_submision in code_submision_elements:
        submission_status = code_submision.find_element(
            By.CSS_SELECTOR, "div[class='col-12 col-md-12 form-group execute_btn'] > span:last-of-type")

        if not submission_status.is_displayed():
            for attempt in range(3):
                try:
                    wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "execute_button"))).click()
                    wait.until(EC.visibility_of(submission_status))
                    time.sleep(1)
                    wait_for_loader_disappearance()
                    break
                except TimeoutException as e:
                    print(f"Click attempt {attempt+1} failed: {e}")
                    time.sleep(1)
            else:
                raise ValueError("Unable to click submit button after 3 retries ❌")

In [ ]:
def complete_course_subtopic(subtopic_number: str, subtopic_name: str, subtopic_questions: list):
    no_of_subtopic_questions = len(subtopic_questions)
    for i in range(no_of_subtopic_questions):
    # for i in range(0,1):
        question = subtopic_questions[i]

        question_number = question.find_element(
            By.CLASS_NAME, "course-question-number").text
        question_name = question.find_element(
            By.CLASS_NAME, "course-question-name").text
        question_type = question.find_element(
            By.CLASS_NAME, "course-question-type").text

        question_status = bool(question.find_elements(
            By.CSS_SELECTOR, ".question-complete-tick-mark-mv > img"))

        if not question_status:
        # if True:
            if question_type == "Coding":
                print(f"Attempting question- {question_name} ({question_type})")

                question.find_element(
                    By.CLASS_NAME, "isQuestionModal").click()

                wait.until(EC.number_of_windows_to_be(2))
                for window in driver.window_handles:
                    if window != original_window:
                        driver.switch_to.window(window)
                        break

                question_file_path = ''
                if question_type == "Tutorial":
                    try:
                        complete_tutorial()
                    except TimeoutException:
                        driver.refresh()
                        wait_for_loader_disappearance()
                        print("Tutorial did not complete on first attempt. Attempting tutorial again.")
                        complete_tutorial()
                else:
                    question_file_path = os.path.join(
                        SOLUTIONS_FOLDER, COURSE_NAME)

                    for folder in os.listdir(question_file_path):
                        if folder.startswith(subtopic_number + ". "):
                            question_file_path = os.path.join(
                                question_file_path, folder)
                            break
                        
                    for folder in os.listdir(question_file_path):
                        if folder.startswith(question_number + ". "):
                            question_file_path = os.path.join(
                                question_file_path, folder)
                            break
                    if question_type != "Coding":
                        question_file_path = os.path.join(
                            question_file_path, "Answer.txt")
                    else:
                        for file in os.listdir(question_file_path):
                            for e in EXTENSION_MAPPING.keys():
                                if file.endswith(e):
                                    question_file_path = os.path.join(
                                        question_file_path, file)
                                    break

                    if question_type == "Coding":
                        try:
                            complete_coding_question(question_file_path)
                        except TimeoutException:
                            driver.refresh()
                            wait_for_loader_disappearance()
                            print("Coding question did not complete on first attempt. Attempting question again.")
                            complete_coding_question(question_file_path)
                    elif question_type == "MCQ":
                        complete_mcq_question(question_file_path)
                    elif question_type == "MQ":
                        complete_textarea_question(question_file_path)
                    else:
                        print(f"Unknown question type- {question_type}")
                        break

                time.sleep(1)  # For the purpose of visual testing
                driver.close()
                driver.switch_to.window(original_window)

    print("Subtopic questions execution complete.")

In [ ]:
"""
# Print course topics

for course_topic in course_topics:
    course_topic_number = course_topic.find_element(
        By.CSS_SELECTOR, ".course-topics-number > p").text
    course_topic_name = course_topic.find_element(
        By.CSS_SELECTOR, ".course-topics-name > p").text
    print(course_topic_number, course_topic_name)
"""


course_topics = driver.find_elements(
    By.CLASS_NAME, "course-topics")
wait_for_loader_disappearance()
time.sleep(2) # For the purpose of visual testing

# t = 1
# for i in range(t-1, t):
for i in range(len(course_topics)):
    course_topic = course_topics[i]

    driver.execute_script(
        "arguments[0].scrollIntoView({block: 'center'});", course_topic)

    subtopic_number = course_topic.find_element(
        By.CSS_SELECTOR, "div.course-topics-number > p").text
    
    is_subtopic_complete = bool(driver.find_elements(By.CLASS_NAME, "icon-1"))
    if is_subtopic_complete:
        continue

    """
    Code to determine whether or not to click on subtopic. Click if not already clicked.
    """
    course_topics_item = driver.find_elements(
        By.CLASS_NAME, "course-topics")
    preclicked_subtopic = -1
    for j in range(len(course_topics_item)):
        item = course_topics_item[j]
        if "clicked" in item.get_attribute("class"):
            preclicked_subtopic = j
            break

    if not i == preclicked_subtopic:
        wait_for_loader_disappearance()
        course_topic.click()
        # print(f"Subtopic number {subtopic_number} clicked.")
    """
    """

    wait_for_loader_disappearance()
    subtopic_name = course_topic.find_element(
        By.CSS_SELECTOR, ".course-topics-name > p").text

    subtopic_questions = driver.find_elements(By.CLASS_NAME, "course-question")
    subtopic_questions = [item for item in subtopic_questions if item.is_displayed()]
    if len(subtopic_questions) == 0:
        raise ValueError("No subtopic questions! ❌")

    print("#" * 30)
    print(f"{subtopic_number}. {subtopic_name}, ({len(subtopic_questions)} questions)")

    no_of_incomplete_questions = 0
    for question in subtopic_questions:
        if not question.find_elements(By.CSS_SELECTOR, "div.question-complete-tick-mark-mv > img"):
            no_of_incomplete_questions += 1
    print(f"{no_of_incomplete_questions}/{len(subtopic_questions)} incomplete questions ❓")
    print("#" * 30, "\n\n")

    complete_course_subtopic(
    subtopic_number, subtopic_name, subtopic_questions)